In [1]:
!pip install split-folders

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import cv2
import os
import PIL
import pathlib
import splitfolders

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-08-11 21:19:27.071940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 21:19:27.072092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 21:19:27.209136: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [5]:
import wandb
from wandb.integration.keras import WandbCallback
from kaggle_secrets import UserSecretsClient


In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [7]:
# Dataset paths
base_ds = '/kaggle/input/potatodataset/Potato Leaf Disease Dataset in Uncontrolled Environment'
base_ds = pathlib.Path(base_ds)

In [8]:
# Prepare the data for Stratified K-Fold
data = []
labels = []

In [9]:
# Gathering file paths and their respective labels
for class_dir in base_ds.glob('*'):
    class_name = class_dir.name
    for img_path in class_dir.glob('*'):
        data.append(str(img_path))
        labels.append(class_name)

In [10]:
# Convert labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Setting up Stratified K-Fold
n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=123)

In [11]:
# Image data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   zoom_range=0.15,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.15,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
!pip install wandb --upgrade

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.17.4
    Uninstalling wandb-0.17.4:
      Successfully uninstalled wandb-0.17.4


In [13]:
import wandb

# Store results from each fold
fold_results = []

# K-Fold Cross-Validation
for fold, (train_idx, val_idx) in enumerate(skf.split(data, labels)):
    print(f"Training fold {fold+1}/{n_splits}...")
    
    # Split data into training and validation sets
    train_files = np.array(data)[train_idx]
    train_labels = np.array(labels)[train_idx]
    val_files = np.array(data)[val_idx]
    val_labels = np.array(labels)[val_idx]
    
    # Convert integer labels back to their string equivalents
    train_labels_str = label_encoder.inverse_transform(train_labels)
    val_labels_str = label_encoder.inverse_transform(val_labels)
    
    # Create training and validation datasets
    train_generator = train_datagen.flow_from_dataframe(
        pd.DataFrame({'filename': train_files, 'class': train_labels_str}),
        x_col='filename',
        y_col='class',
        target_size=(128, 128),
        class_mode='categorical',
        batch_size=16,
        shuffle=True
    )
    
    val_generator = train_datagen.flow_from_dataframe(
        pd.DataFrame({'filename': val_files, 'class': val_labels_str}),
        x_col='filename',
        y_col='class',
        target_size=(128, 128),
        class_mode='categorical',
        batch_size=16,
        shuffle=False
    )
    
    # Model configuration
    CONFIG = dict(
        batch_size=16,
        img_height=128,
        img_width=128,
    )
    
    # Rename the model variable to avoid conflicts
    densenet_model = tf.keras.applications.DenseNet201(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
    densenet_model.trainable = True
    inputs = tf.keras.Input((128, 128, 3))
    x = densenet_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(x)
    densenet_model = tf.keras.Model(inputs, x)
    
    densenet_model.compile(optimizer=keras.optimizers.Adam(1e-5),
                           loss='categorical_crossentropy', metrics=['accuracy'])
    
    # W&B run
    run = wandb.init(settings=wandb.Settings(start_method="fork"),
                     reinit=True,
                     project='Preet_Potato_kfold',
                     entity="pritmhala03",
                     config=CONFIG,
                     group='DenseNet201',
                     job_type='train')
    
    wandb.config.type = f'fold_{fold+1}'
    
    # Training with manual logging
    for epoch in range(20):
        history = densenet_model.fit(
            train_generator,
            epochs=1,
            validation_data=val_generator
        )
        
        # Log metrics manually to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": history.history['loss'][0],
            "train_accuracy": history.history['accuracy'][0],
            "val_loss": history.history['val_loss'][0],
            "val_accuracy": history.history['val_accuracy'][0]
        })
    
    # Evaluate model on validation set and log final metrics
    val_loss, val_accuracy = densenet_model.evaluate(val_generator)
    print(f"Fold {fold+1} - Loss: {val_loss}, Accuracy: {val_accuracy * 100:.2f}%")
    fold_results.append((val_loss, val_accuracy))
    wandb.log({"final_val_loss": val_loss, "final_val_accuracy": val_accuracy})
    
    # Save the model in TensorFlow SavedModel format
    # densenet_model.save(f'DenseNet201_fold_{fold+1}', save_format='tf')
    
    # Finish W&B run
    run.finish()

# After all folds
for i, (val_loss, val_accuracy) in enumerate(fold_results):
    print(f"Fold {i+1} - Loss: {val_loss}, Accuracy: {val_accuracy * 100:.2f}%")


Training fold 1/5...
Found 2460 validated image filenames belonging to 7 classes.
Found 616 validated image filenames belonging to 7 classes.
74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1723411500.660273     117 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 870ms/step - accuracy: 0.2684 - loss: 1.7870

W0000 00:00:1723417193.854547     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


154/154 ━━━━━━━━━━━━━━━━━━━━ 372s 1s/step - accuracy: 0.2689 - loss: 1.7863 - val_accuracy: 0.4049 - val_loss: 1.5222
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 288ms/step - accuracy: 0.4792 - loss: 1.3897 - val_accuracy: 0.5593 - val_loss: 1.2193
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 290ms/step - accuracy: 0.5979 - loss: 1.1326 - val_accuracy: 0.6195 - val_loss: 1.0286
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 288ms/step - accuracy: 0.6481 - loss: 0.9697 - val_accuracy: 0.6537 - val_loss: 0.8821
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 286ms/step - accuracy: 0.7142 - loss: 0.8153 - val_accuracy: 0.6764 - val_loss: 0.8471
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 287ms/step - accuracy: 0.7483 - loss: 0.7353 - val_accuracy: 0.7398 - val_loss: 0.7552
154/154 ━━━━━━━━━━━━━━━━━━━━ 47s 292ms/step - accuracy: 0.7586 - loss: 0.6738 - val_accuracy: 0.7528 - val_loss: 0.7132
154/154 ━━━━━━━━━━━━━━━━━━━━ 46s 291ms/step - accuracy: 0.7774 - loss: 0.6313 - val_accuracy: 0.7415 - val_loss: 0.6934
154/154 ━━━━━━━━━━━━━━━━━━━━ 45s 284ms/ste

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
final_val_accuracy,▁
final_val_loss,▁
train_accuracy,▁▃▄▅▅▆▆▆▇▇▇▇▇▇██████
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▄▅▅▆▇▇▇▇██▇████████
val_loss,█▆▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,20
final_val_accuracy,0.82439
final_val_loss,0.52703
train_accuracy,0.92564


Fold 1 - Loss: 0.5938485264778137, Accuracy: 78.90%
Fold 2 - Loss: 0.5092435479164124, Accuracy: 82.44%
Fold 3 - Loss: 0.5398438572883606, Accuracy: 81.30%
Fold 4 - Loss: 0.5189046263694763, Accuracy: 81.46%
Fold 5 - Loss: 0.5270301103591919, Accuracy: 82.44%


In [14]:
# After all folds
for i, (val_loss, val_accuracy) in enumerate(fold_results):
    print(f"Fold {i+1} - Loss: {val_loss}, Accuracy: {val_accuracy * 100:.2f}%")

Fold 1 - Loss: 0.5938485264778137, Accuracy: 78.90%
Fold 2 - Loss: 0.5092435479164124, Accuracy: 82.44%
Fold 3 - Loss: 0.5398438572883606, Accuracy: 81.30%
Fold 4 - Loss: 0.5189046263694763, Accuracy: 81.46%
Fold 5 - Loss: 0.5270301103591919, Accuracy: 82.44%


Fold 1 - Loss: 0.5938485264778137, Accuracy: 78.90%

Fold 2 - Loss: 0.5092435479164124, Accuracy: 82.44%

Fold 3 - Loss: 0.5398438572883606, Accuracy: 81.30%

Fold 4 - Loss: 0.5189046263694763, Accuracy: 81.46%

Fold 5 - Loss: 0.5270301103591919, Accuracy: 82.44%

In [15]:
# After all folds: Compute average performance
average_val_loss = np.mean([result[0] for result in fold_results])
average_val_accuracy = np.mean([result[1] for result in fold_results])
print(f"Average Validation Loss: {average_val_loss}, Average Validation Accuracy: {average_val_accuracy * 100:.2f}%")

Average Validation Loss: 0.537774133682251, Average Validation Accuracy: 81.31%


Average Validation Loss: 0.537774133682251,

Average Validation Accuracy: 81.31%